## Process data

In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload 2

<IPython.core.display.Javascript object>

In [3]:
import pandas as pd

df = pd.read_pickle("processed_df.pkl")

<IPython.core.display.Javascript object>

In [21]:
analyze_col = "tasks"

<IPython.core.display.Javascript object>

In [5]:
import texthero as hero
import scattertext as st

df[analyze_col] = (
    df[analyze_col]
    .pipe(hero.remove_punctuation)
    .pipe(hero.remove_digits)
    .pipe(hero.fillna)
    .pipe(hero.remove_stopwords)
    .pipe(hero.remove_whitespace)
)

<IPython.core.display.Javascript object>

## Terms

In [6]:
import re

<IPython.core.display.Javascript object>

In [7]:
filtered_df = df[~df[analyze_col].isna()][["title", analyze_col, "page_host"]]

<IPython.core.display.Javascript object>

In [8]:
filtered_df[analyze_col] = filtered_df[analyze_col].apply(
    lambda row: re.sub(r"(?<=[.,])(?=[^\s])", r" ", row)
)

<IPython.core.display.Javascript object>

In [9]:
filtered_df["parse"] = filtered_df[analyze_col].apply(st.whitespace_nlp_with_sentences)

<IPython.core.display.Javascript object>

In [10]:
corpus = (
    st.CorpusFromParsedDocuments(filtered_df, category_col="title", parsed_col="parse")
    .build()
    .get_unigram_corpus()
    .compact(st.AssociationCompactor(2000))
)

<IPython.core.display.Javascript object>

In [11]:
html = st.produce_scattertext_explorer(
    corpus,
    category="data scientist",
    category_name="Data scientist",
    not_category_name="Data Engineer",
    minimum_term_frequency=5,
    pmi_threshold_coefficient=0,
    width_in_pixels=1000,
    metadata=corpus.get_df()["page_host"],
    transform=st.Scalers.dense_rank,
)
open("data_science_vs_data_engineer_terms_task.html", "w").write(html)

1608035

<IPython.core.display.Javascript object>

## Phrases

In [12]:
import pytextrank, spacy
import scattertext as st

nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("textrank", last=True)

<IPython.core.display.Javascript object>

In [13]:
filtered_df = filtered_df.assign(
    parse=lambda df: df[analyze_col].apply(nlp),
    position=lambda df: df.title.apply(
        {"data scientist": "Data scientist", "data engineer": "Data engineer"}.get
    ),
)

<IPython.core.display.Javascript object>

In [14]:
corpus = (
    st.CorpusFromParsedDocuments(
        filtered_df,
        category_col="title",
        parsed_col="parse",
        feats_from_spacy_doc=st.PyTextRankPhrases(),
    )
    .build()
    .get_unigram_corpus()
    .compact(st.AssociationCompactor(2000, use_non_text_features=True))
)

/home/user/Data-science/venv/lib/python3.8/site-packages/scattertext/termscoring/ScaledFScore.py:129: RuntimeWarning: invalid value encountered in true_divide
  precision = (cat_word_counts * 1. / (cat_word_counts + not_cat_word_counts))
/home/user/Data-science/venv/lib/python3.8/site-packages/scipy/stats/_distn_infrastructure.py:1847: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= np.asarray(_b)) & cond0


<IPython.core.display.Javascript object>

In [15]:
term_category_scores = corpus.get_metadata_freq_df("")
print(term_category_scores)

                                                    data scientist  \
term                                                                 
Bachelor                                                  0.633226   
Explain                                                   0.196449   
relationships                                             0.551930   
Improve                                                   0.159983   
SQL                                                       1.171628   
...                                                            ...   
Participate management database                           0.000000   
BigData Hadoop Spark Yarn Oozie Scala Agile TDD...        0.000000   
NoSQL Cloud                                               0.000000   
Microsoft Azure Architect                                 0.000000   
Atlanta GA Full time Design                               0.000000   

                                                    data engineer  
term                 

<IPython.core.display.Javascript object>

In [16]:
import numpy as np

<IPython.core.display.Javascript object>

In [17]:
term_ranks = np.argsort(np.argsort(-term_category_scores, axis=0), axis=0) + 1
metadata_descriptions = {
    term: '<br/>' + '<br/>'.join(
        '<b>%s</b> TextRank score rank: %s/%s' % (cat, term_ranks.loc[term, cat], corpus.get_num_metadata())
        for cat in corpus.get_categories())
    for term in corpus.get_metadata()
}


<IPython.core.display.Javascript object>

In [18]:
category_specific_prominence = term_category_scores.apply(
    lambda r: r['data scientist'] if r['data scientist'] > r['data engineer'] else -r['data engineer'],
    axis=1
)


<IPython.core.display.Javascript object>

In [19]:
html = st.produce_scattertext_explorer(
    corpus,
    category="data scientist",
    category_name="Data scientist",
    not_category_name="Data Engineer",
    minimum_term_frequency=0,
    pmi_threshold_coefficient=0,
    width_in_pixels=1000,
    transform=st.Scalers.dense_rank,
    metadata=corpus.get_df()["page_host"],
    scores=category_specific_prominence,
    sort_by_dist=False,
    use_non_text_features=True,
    topic_model_term_lists={term: [term] for term in corpus.get_metadata()},
    topic_model_preview_size=0,
    metadata_descriptions=metadata_descriptions,
    use_full_doc=True,
)

<IPython.core.display.Javascript object>

In [20]:
open("./data_scientist_vs_data_engineer_task.html", "w").write(html)

2501920

<IPython.core.display.Javascript object>